In [1]:
import sys
import os

sys.path.append("..")

from pytesseract import Output
import pytesseract
import cv2
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import numpy as np
from tqdm import tqdm
from vid2frames import convert2frames

In [4]:
# First run vid2frames on the video I'll be dugging OCR on

def reset_frames(input_path: str, output_folder: str):
    if os.path.exists(output_folder):
        os.system(f"rm -rf {output_folder}")
    if not os.path.exists(output_folder):
        os.mkdir(output_folder)
        os.mkdir(output_folder + "/images")
    convert2frames(input_path, output_folder + "/images")

reset_frames("../videos/artisanal/play_ninety.mp4", "../frames/play_ninety")

100%|██████████| 578/578 [00:41<00:00, 14.07it/s]


In [7]:
# Don't bother with the dataset and the dataloader just yet

def draw_basic_result(fin: str, min_conf: float = 0.95):
    image = cv2.imread(fin)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = pytesseract.image_to_data(rgb, output_type=Output.DICT)
    print(results)
    
    for i in range(0, len(results["text"])):
        # Get the box
        x = results["left"][i]
        y = results["top"][i]
        w = results["width"][i]
        h = results["height"][i]
        
        # Text and confidence
        text = results["text"][i]
        conf = int(results["conf"][i])
        
        # Draw on the image
        if conf > min_conf:
            text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(image, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX,
                1.2, (0, 0, 255), 3)
            
    # Save it to see quality 
    cv2.imwrite("temp.jpg", image)

draw_basic_result("../frames/play_ninety/images/f_162000.jpg")

{'level': [1, 2, 3, 4, 5], 'page_num': [1, 1, 1, 1, 1], 'block_num': [0, 1, 1, 1, 1], 'par_num': [0, 0, 1, 1, 1], 'line_num': [0, 0, 0, 1, 1], 'word_num': [0, 0, 0, 0, 1], 'left': [0, 85, 85, 85, 85], 'top': [0, 0, 0, 0, 0], 'width': [640, 473, 473, 473, 473], 'height': [360, 360, 360, 360, 360], 'conf': [-1, -1, -1, -1, 95], 'text': ['', '', '', '', '']}


In [8]:
import keras_ocr
import matplotlib.pyplot as plt

# keras-ocr will automatically download pretrained
# weights for the detector and recognizer.
pipeline = keras_ocr.pipeline.Pipeline()

# Get a set of three example images
images = [
    keras_ocr.tools.read(url) for url in [
        'https://upload.wikimedia.org/wikipedia/commons/b/bd/Army_Reserves_Recruitment_Banner_MOD_45156284.jpg',
        'https://upload.wikimedia.org/wikipedia/commons/e/e8/FseeG2QeLXo.jpg',
        'https://upload.wikimedia.org/wikipedia/commons/b/b4/EUBanana-500x112.jpg'
    ]
]

# Each list of predictions in prediction_groups is a list of
# (word, box) tuples.
prediction_groups = pipeline.recognize(images)

# Plot the predictions
fig, axs = plt.subplots(nrows=len(images), figsize=(20, 20))
for ax, image, predictions in zip(axs, images, prediction_groups):
    keras_ocr.tools.drawAnnotations(image=image, predictions=predictions, ax=ax)

ModuleNotFoundError: No module named 'tensorflow'

![Alt Result](temp.jpg)